In [2]:
!tensorboard --version

1.14.0


In [3]:
!tensorboard --logdir=runs

TensorBoard 1.14.0 at http://LIVVJ0V9J1WV:6006/ (Press CTRL+C to quit)
^C


In [8]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter


torch.set_printoptions(linewidth=120)


In [6]:
train_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train=True,
    download=True,
    transform=transforms.Compose(
    [transforms.ToTensor()]))

train_loader = torch.utils.data.DataLoader(train_set, batch_size=16)

In [7]:
class Network(nn.Module):
    def __init__(self):
        #super(Network, self).__init__()
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
        ## implement forward pass
        t = t
        
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = t.reshape(-1, 12*4*4)
        t = self.fc1(t)
        t = F.relu(t)
        
        t = self.fc2(t)
        t = F.relu(t)
        
        t = self.out(t)
        #t = F.softmax(t, dim=1)
        
        return t

In [10]:
tb = SummaryWriter()

network = Network()
images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb.add_image('images', grid)
tb.add_graph(network, images)
tb.close()

In [12]:
import torch.optim as optim

optimizer = optim.Adam(network.parameters(), lr=0.01)
tb = SummaryWriter()

for epoch in range(5):
    total_loss = 0
    total_correct = 0

    for batch in train_loader:
        images, labels = batch

        preds = network(images)
        loss = F.cross_entropy(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss+=loss.item()
        total_correct+=preds.argmax(dim=1).eq(labels).sum().item()
    print (epoch, total_loss, total_correct)
    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Num correct', total_correct, epoch)
    
    tb.add_histogram('conv1 w', network.conv1.weight, epoch)

0 2760.169764354825 43007
1 2220.3502415157855 46645
2 2118.3734463974833 47409
3 2079.9158128499985 47899
4 2040.9862843081355 47985
